---
title: "3 - Funciones de orden superior"
toc: true
---

## Introducción

Las funciones de orden superior son una herramienta muy importante en la programación funcional.
A lo largo de esta unidad, trabajaremos con las siguientes variedades de funciones de orden superior:

* Funciones que aceptan funciones como argumentos.
* Funciones que devuelven una función como resultado.
* Funciones que aceptan funciones como argumentos y devuelven una función como resultado.

En la Sección [Funciones de orden superior](./01_paradigma.ipynb#sec-orden) de [El paradigma funcional](./01_paradigma.ipynb) nos inciamos en el trabajo con funciones que aceptan funciones como argumento y funciones que devuelven funciones como resultado.

En este capítulo, vamos a enfocarnos las funciones de orden superior más elementales: `map`, `filter` y `reduce`. Las primeras dos están disponibles por defecto en nuestra sesión de Python (ya que son funciones _built-in_), mientras que a `reduce` la tenemos que importar desde el módulo estándar `functools`.

Las funciones `map`, `filter` y `reduce` son funciones de orden superior fundamentales en la programación funcional. Actúan como primitivas básicas para procesar y transformar secuencias, y muchas otras operaciones funcionales pueden construirse a partir de ellas o expresarse en términos de estas.

## _Map_

Supongamos que tenemos una secuencia de palabras y queremos invertir el orden de los caracteres de cada una.
Para ello, vamos a rebanadar cada cadena desde el principio al final usando un paso de `-1`. Por ejemplo:

In [1]:
"cosa"[::-1]

'asoc'

Si quisiéramos obtener una lista con las palabras invertidas, podríamos crear una nueva lista, recorrer la original con un bucle `for`, invertir cada palabra y guardarla en la lista nueva.

In [2]:
palabras = ["hola", "mate", "somos"," libro", "conocer", "anilina", "programa"]
palabras_invertidas = []

for palabra in palabras:
    palabras_invertidas.append(palabra[::-1])

print("Palabras originales:", palabras, "\n", sep="\n")
print("Palabras invertidas:", palabras_invertidas, sep="\n")

Palabras originales:
['hola', 'mate', 'somos', ' libro', 'conocer', 'anilina', 'programa']


Palabras invertidas:
['aloh', 'etam', 'somos', 'orbil ', 'reconoc', 'anilina', 'amargorp']


La alternativa funcional consiste en utilizar `map` para **aplicar una función a cada palabra de la secuencia**.
En este caso, aplicamos la función `invertir`, que invierte los caracteres de una palabra, a cada elemento de la lista `palabras`.

In [3]:
def invertir(x):
    return x[::-1]

list(map(invertir, palabras))

['aloh', 'etam', 'somos', 'orbil ', 'reconoc', 'anilina', 'amargorp']

Así, se obtiene una nueva lista con las palabras invertidas, sin necesidad de iterar manualmente con un bucle `for`.

Si quisiéramos que el programa fuese aún más conciso, podríamos usar una función anónima en vez de una función regular:

In [4]:
list(map(lambda x: x[::-1], palabras))

['aloh', 'etam', 'somos', 'orbil ', 'reconoc', 'anilina', 'amargorp']

::: {.callout-note}
##### El objeto `map`

En el ejemplo anterior usamos `list` para convertir el resultado de `map` en una lista.
Este paso, que puede parecer innecesario, es fundamental si queremos obtener una lista como resultado final.
De lo contrario, la llamada a `map` devuelve un objeto de tipo `map`.

```python
map(lambda x: x[::-1], palabras)
```
```cmd
<map object at 0x7fd2fc1ad360>
```

Este objeto, **perezoso** e **iterable**, puede recorrerse o convertirse en otras colecciones como listas, tuplas o conjuntos:

```python
list(map(lambda x: x[::-1], palabras))
```
```cmd
['aloh', 'etam', 'somos', 'orbil ', 'reconoc', 'anilina', 'amargorp']
```
:::

### Estructuras de datos complejas

En el ejemplo anterior se usó `map` sobre una secuencia simple de cadenas de texto. Sin embargo, eso no implica que su uso se limite a casos sencillos.

Supongamos ahora que tenemos una lista anidada de números, es decir, una lista que contiene otras listas con valores numéricos:

In [5]:
ventas = [
    [22.5, 9.3, 11.0],
    [5.4, 22.5],
    [3.0, 3.0, 12.9, 7.5],
]

Si queremos calcular el total de cada sublista, podemos combinar `map` con la función `sum`. Esto aplica `sum` a cada elemento de la lista `ventas`, generando como resultado una nueva lista con los totales de cada sublista.

In [6]:
list(map(sum, ventas))

[42.8, 27.9, 26.4]

De manera similar, se puede obtener el mínimo, el máximo, la media u otra medida de interés aplicando la función correspondiente a cada sublista.

Usando una combinación más compleja de `map`s y expresiones `lambda`, se puede determinar cuáles sublistas de `ventas` contienen al menos un valor mayor a 20.

In [7]:
list(
    map(
        lambda sublista: any(map(lambda x: x > 20,  sublista)),
        ventas
    )
)

[True, True, False]

Como puede observarse, un programa que utiliza `map` junto con expresiones `lambda` puede volverse difícil de leer y comprender rápidamente, especialmente a medida que la lógica se vuelve más compleja.

Para finalizar este listado de ejemplos, observemos uno donde se crea un diccionario a partir del `map`, en vez de una lista. 

Debajo, tenemos una lista con diccionarios. Cada diccionario contiene el nombre y las notas de una persona.
Nuestro objetivo es obtener un nuevo diccionario que tenga por claves al nombre de la persona, y por valor a la nota promedio.

In [8]:
notas = [
    {
        "nombre": "Mariano",
        "notas": [6, 9, 9, 8]
    },
    {
        "nombre": "Daniela",
        "notas": [6, 7, 7, 8]
    },
    {
        "nombre": "Sofía",
        "notas": [8, 6, 9, 8]
    },
]

Sin utilizar un enfoque funcional, una solución posible es la siguiente:

In [9]:
def media(x):
    return sum(x) / len(x)

promedios = {}

for datum in notas:
    promedios[datum["nombre"]] = media(datum["notas"])

promedios

{'Mariano': 8.0, 'Daniela': 7.0, 'Sofía': 7.75}

En cambio, utilizando `map`:

In [10]:
dict(map(lambda datum: (datum["nombre"], media(datum["notas"])), notas))

{'Mariano': 8.0, 'Daniela': 7.0, 'Sofía': 7.75}

La clave está en notar que la expresión `lambda` **devuelve una tupla de dos elementos**, donde el primero es el nombre y el segundo, la nota promedio.
A partir de estos pares `(str, float)`, se puede construir directamente un diccionario.

### Usando `map` con múltiples iterabes

Hasta ahora hemos utilizado `map` con funciones que se aplican sobre los elementos de un único iterable.
Sin embargo, `map` también acepta múltiples iterables y los recorre en paralelo, lo que la convierte en una **función variádica**.
Esto permite aplicar funciones que toman más de un argumento, una vez por cada grupo de elementos correspondientes.

Supongamos que queremos redondear un listado de números utilizando diferentes niveles de precisión. Para redondear un único número podemos usar directamente `round`:

In [11]:
round(29.12951138, 4)

29.1295

Si quisiéramos redonear múltiples números en una lista, usando el mismo nivel de precisión, podemos usar `map` y `round`:

In [12]:
numeros = [
    30.60726375,
    78.12297368,
    61.94972186,
    68.78842783,
    55.60016942,
    94.9760221,
    90.41151716,
    38.72727347,
    21.30193307,
    66.39407577
]
list(map(lambda x: round(x, 3), numeros))

[30.607, 78.123, 61.95, 68.788, 55.6, 94.976, 90.412, 38.727, 21.302, 66.394]

¿Y si quisiéramos aplicar diferentes niveles de precisión a cada número? Para ello, **también podemos usar `map`**.
Definimos una función que reciba dos argumentos y luego iteramos en paralelo sobre dos iterables: uno con los números y otro con las precisiones correspondientes.

In [13]:
precisiones = [2, 2, 3, 3, 4, 4, 5, 5, 2, 2]
list(map(lambda x, y: round(x, y), numeros, precisiones))

[30.61, 78.12, 61.95, 68.788, 55.6002, 94.976, 90.41152, 38.72727, 21.3, 66.39]

::: {.callout-note}
##### ¿Qué pasa si un iterable es más corto que el otro? 🤔

Cuando se recorren múltiples iterables con `map`, la iteración se detiene tan pronto como se agota el iterable más corto.
Por ejemplo, si tenemos 10 números pero solo 5 precisiones, `map` aplicará la función únicamente a los primeros 5 pares de elementos:

```python
precisiones = [1, 2, 3, 4, 5]
list(map(lambda x, y: round(x, y), numeros, precisiones))
```
```cmd
[30.6, 78.12, 61.95, 68.7884, 55.60017]
```
:::

## _Filter_

`filter` se utiliza para seleccionar —o, más precisamente, filtrar— elementos de un iterable según el resultado de aplicar una función.
A diferencia de `map`, la función usada por `filter` se aplica sobre los elementos de **un solo iterable** y **debe devolver un valor booleano**.
El resultado es un nuevo iterable que contiene únicamente los elementos para los que la función retorna `True`.

Como ejemplo del uso de `filter`, vamos a seleccionar las notas menores a 6 a partir de una lista de calificaciones.

In [14]:
notas = [6, 9, 6, 5, 7, 4, 5, 8, 3, 10, 9, 4, 7, 8]
list(filter(lambda x: x < 6, notas))

[5, 4, 5, 3, 4]

De este modo, resulta sencillo calcular el promedio de las notas de aquellos que no aprobaron:

In [15]:
media(list(filter(lambda x: x < 6, notas)))

4.2

Retomando el ejemplo del listado de palabras que se querían revertir, se podría usar `filter` para seleccionar solo aquellas palabras que sean palíndromos, es decir, capicúa.

In [16]:
palabras = ["hola", "mate", "somos"," libro", "conocer", "anilina", "programa"]
capicuas = list(filter(lambda p: p[::-1] == p, palabras))
capicuas

['somos', 'anilina']

Naturalmente, `filter` también puede utilizarse para filtrar objetos más complejos.
Por ejemplo, si tenemos una lista de diccionarios con información de estudiantes (nombre, ciudad de origen, edad y fecha de inscripción),
podemos usar `filter` para seleccionar aquellos que cumplan **una o más condiciones**.
En ese caso, el valor booleano que devuelve la función se construye combinando condiciones mediante operadores lógicos como `and`.

In [17]:
datos = [
    {"nombre": "Agustina", "ciudad": "Casilda", "edad": 18, "inscripcion": 2025},
    {"nombre": "Emiliano", "ciudad": "Rosario", "edad": 21, "inscripcion": 2024},
    {"nombre": "David", "ciudad": "Pergamino", "edad": 19, "inscripcion": 2024},
    {"nombre": "Julieta", "ciudad": "Rosario", "edad": 19, "inscripcion": 2025},
    {"nombre": "Victoria", "ciudad": "Chañar Ladeado", "edad": 18, "inscripcion": 2025},
    {"nombre": "Fernando", "ciudad": "Rosario", "edad": 20, "inscripcion": 2024},
    {"nombre": "Mateo", "ciudad": "Pérez", "edad": 23, "inscripcion": 2025},
    {"nombre": "Lucía", "ciudad": "Rosario", "edad": 22, "inscripcion": 2022},
    {"nombre": "Joaquín", "ciudad": "Casilda", "edad": 19, "inscripcion": 2025},
    {"nombre": "Micaela", "ciudad": "Rosario", "edad": 18, "inscripcion": 2024},
]

list(filter(lambda x: x["ciudad"] == "Rosario" and x["inscripcion"] == 2025, datos))

[{'nombre': 'Julieta', 'ciudad': 'Rosario', 'edad': 19, 'inscripcion': 2025}]

## _Reduce_

La función `reduce` permite **reducir** una secuencia a un único valor aplicando de forma sucesiva una de dos argumentos sobre sus elementos.

Para utilizarla, es necesario importarla desde el módulo estándar `functools`:

In [18]:
from functools import reduce

`reduce` aplica la función acumulando resultados de a pares, desde el primer elemento hasta el último. Por ejemplo:

```python
reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])
```

equivale a:

```python
((((1 + 2) + 3) + 4) + 5)
```

En este caso, es simplemente una forma más rebuscada de escribir `sum([1, 2, 3, 4, 5])` en Python.

Para entender cómo funciona el proceso de acumulación en `reduce`, podemos definir una función que imprima los valores de sus argumentos en cada paso:

In [19]:
def sumar(x, y):
    print(f"x={x}, y={y}")
    return x + y

reduce(sumar, [1, 2, 3, 4, 5])

x=1, y=2
x=3, y=3
x=6, y=4
x=10, y=5


15

En la primera llamada, `x` e `y` son los dos primeros elementos de la secuencia.
En la segunda, `x` es el resultado de la llamada anterior, y `y` es el siguiente elemento de la secuencia.
Este proceso continúa hasta que se recorre toda la lista.

En resumen:

* `x` representa el valor acumulado hasta el momento.
* `y` es el nuevo elemento a combinar.

Así, `reduce` va aplicando la función paso a paso, acumulando resultados hasta obtener un único valor final.

Muchas operaciones comunes, como sumas, productos, mínimos o máximos, pueden expresarse mediante reducciones.
Por ejemplo, es posible calcular el factorial de un número utilizando una `reduce`:

In [20]:
def factorial(n):
    return reduce(lambda x, y: x * y, range(1, n + 1))

factorial(5)

120

La reducción mediante la multiplicación de dos números, aplicada a la secuencia del 1 al `n`, da como resultado el factorial de `n`.

Finalmente, podemos ver que combinando una función que devuelve el mayor de dos números y una reducción, es posible obtener el máximo de una secuencia.

In [21]:
def mayor(x, y):
    if x > y:
        return x
    return y

reduce(mayor, [23, 49, 6, 32, 101, 9])

101

Vale la pena mencionar que `reduce` acepta un tercer argumento opcional, que especifica el valor inicial de la reducción.
Este valor se utiliza como punto de partida antes de procesar los elementos del iterable.

In [22]:
def sumar(x, y):
    print(f"x={x}, y={y}")
    return x + y

reduce(sumar, [1, 2, 3, 4, 5], 20)

x=20, y=1
x=21, y=2
x=23, y=3
x=26, y=4
x=30, y=5


35

::: {.callout-note}
##### Expresiones condicionales 🔀😱

La reducción anterior puede expresarse de forma más concisa utilizando **expresiones condicionales**:

```python
reduce(lambda x, y: x if x > y else y, [23, 49, 6, 32, 101, 9])
```

Estas expresiones permiten simplificar asignaciones condicionales.
Por ejemplo, el siguiente bloque:

```python
if x > y:
    valor = x
else:
    valor = y
```

puede escribirse de manera más compacta así:

```python
valor = x if x > y else y
```

En términos generales, la sintaxis es:

```python
<valor_si_verdadero> if <condición> else <valor_si_falso>
```

:::

## Resumen de _Map_, _Filter_ y _Reduce_

El siguiente bloque de código resume el funcionamiento de `map`, `filter` y `reduce`.

In [23]:
numeros = [1, 2, 3, 4, 5]

# Map: Aplicar una función a cada elemento de un iterable
cuadrados = list(map(lambda x: x**2, numeros))
print("Map [cuadrados]:", cuadrados)

# Filter: Devuelve el subconjunto de elementos para los que la función devuelve True
pares = list(filter(lambda x: x % 2 == 0, numeros))
print("Filer [pares]:", cuadrados)

# Reduce: Aplica una función de dos argumentos de manera acumulativa a los elementos de una secuencia
producto = reduce(lambda x, y: x * y, numeros)
print("Reduce [producto]:", producto)

Map [cuadrados]: [1, 4, 9, 16, 25]
Filer [pares]: [1, 4, 9, 16, 25]
Reduce [producto]: 120


### Map y Filter como casos particulares de Reduce 😱

Por otro lado, algo menos evidente es que tanto `map` como `filter` pueden verse como casos particulares de `reduce`.

Esta aplicación de `map`:

In [24]:
list(map(lambda x: x * 2, range(10)))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

Puede ser reproducida con el siguiente uso de `reduce`:

In [25]:
def duplicar(x):
    return x * 2

reduce(lambda seq, x: seq + [duplicar(x)], range(10), [])

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

Y el siguiente uso de `filter`

In [26]:
list(filter(lambda x: x % 2 == 1, range(10)))

[1, 3, 5, 7, 9]

In [27]:
def es_impar(x):
    return x % 2 == 1

reduce(lambda seq, x: seq + [x] if es_impar(x) else seq, range(10), [])

[1, 3, 5, 7, 9]

Estas expresiones con `reduce()` son complejas, pero ilustran claramente el poder de la función:
cualquier operación que pueda definirse a partir de una combinación sucesiva de elementos puede, al menos en principio, expresarse como una reducción,
aunque no siempre sea la forma más clara o recomendada de hacerlo.

## _Comprehensions_

Cuando usamos `map` y `filter` obtenemos objetos especiales: `map` devuelve un objeto de tipo `map`, y `filter` devuelve un objeto de tipo `filter`.
Estos objetos son iterables y perezosos, lo que significa que no realizan ninguna operación hasta que se los recorre o convierte en una colección, como una lista.
Por eso, si queremos ver directamente el resultado de una transformación o filtrado, necesitamos envolverlos con `list()`:

```python
numeros = [1, 2, 3]
list(map(lambda x: x * 2, numeros))          # → [2, 4, 6]
list(filter(lambda x: x % 2 == 0, numeros))  # → [2]
```

Aunque `map` y `filter` siguen siendo completamente válidos y útiles, hoy en día se consideran formas algo anticuadas o menos idiomáticas de construir listas transformadas o filtradas en Python.

La alternativa moderna y, en general preferida, son las **comprensiones de listas** (del inglés, _list comprehensions_), que permiten expresar las mismas ideas de forma más clara y legible:

In [28]:
numeros = list(range(11))
[x * 2 for x in numeros] # Reemplaza a list(map())

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [29]:
[x for x in numeros if x % 2 == 0] # Reemplaza a list(filter())

[0, 2, 4, 6, 8, 10]

### _Comprehension_ como reemplazo de _Map_

Supongamos que tenemos una lista de números y queremos restarles su media.

Una forma de hacerlo utilizando un bucle `for` sería:

In [30]:
vector = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

media = media(vector)
vector_centrado = []
for x in vector:
    vector_centrado.append(x - media)

vector_centrado

[-4.5, -3.5, -2.5, -1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5]

Si, en cambio, decidimos usar `map`, podemos hacer:

In [31]:
list(map(lambda x: x - media, vector))

[-4.5, -3.5, -2.5, -1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5]

Finalmente, se puede obtener el mismo resultado usando una _list comprehension_:

In [32]:
[x - media for x in vector]

[-4.5, -3.5, -2.5, -1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5]

La sintaxis general de una  _list comprehension_ que aplica una transformación sobre los elementos de un iterable es:

```python
[<expresión> for elemento in iterable]
```

Como se observa en el ejemplo anterior, lo que aparece en la parte izquierda como `<expresión>` no tiene por qué ser una llamada a una función;
puede ser **cualquier expresión válida** que produzca un resultado.
Es decir, una operación matemática, un formateo de texto, la construcción de una estructura de datos, una llamada a una función, etc.

### _Comprehension_ como reemplazo de _Filter_

Ahora veamos con mayor detalle cómo funciona una _list comprehension_ que reemplaza al uso de `filter`. Para eso, retomemos el ejemplo de las palabras capicúa.

In [33]:
palabras = ["hola", "mate", "somos"," libro", "conocer", "anilina", "programa"]

Inicialmente, podemos construir un listado de palabras capicúa usando un bucle `for`.

In [34]:
capicuas = []
for palabra in palabras:
    if palabra == palabra[::-1]:
        capicuas.append(palabra)
capicuas

['somos', 'anilina']

Luego, podemos construir el listado de palabras capicúa usando la función de orden superior `filter`.

In [35]:
list(filter(lambda x: x == x[::-1], palabras))

['somos', 'anilina']

Y finalmente, se puede obtener exactamente el mismo resultado mediante una _list comprehension_.

In [36]:
[palabra for palabra in palabras if palabra == palabra[::-1]]

['somos', 'anilina']

La sintaxis general de una _list comprehension_ que filtra los elementos de un iterable es:

```python
[elemento for elemento in iterable if <expresión_lógica>]
```

Al igual que en la _list comprehension_ que aplica funciones a todos los ementos, `<expresión_lógica>` puede ser **cualquier expresión** de Python que devuelva un valor `True` o `False`, o que pueda interpretarse como tal.

::: {.callout-note}
##### _Comprehensions_ con expresiones condicionales 😱

La estructura general:

```python
[elemento for elemento in iterable if <expresión_lógica>]
```

puede modificarse cuando se desea evaluar una expresión en caso de que se cumpla una condición y otra distinta si no se cumple.
Para ello, se usa una expresión condicional directamente en la parte izquierda de la comprensión:

```python
[<expresión_si_verdadero> if <condición> else <expresión_si_falso> for elemento in iterable]
```

Por ejemplo:

```python
numeros = [1, 2, 3, 4, 5]
[f"{x} es par" if x % 2 == 0 else f"{x} es impar" for x in numeros]
```
```cmd
['1 es impar', '2 es par', '3 es impar', '4 es par', '5 es impar']
```
:::

::: {.callout-note}
##### _Dictionary comprehensions_ 😱😱

Las comprensiones en Python no están limitadas a listas.
Este patrón también puede utilizarse para construir otras estructuras de datos como diccionarios**, conjuntos e incluso generadores (estructura que veremos más adelante).

Por ejemplo, una **comprensión de diccionario** permite crear un `dict` a partir de una secuencia de pares clave-valor:

```python
def media(x):
    return sum(x) / len(x)

datos = [
    ("Marcos", (4, 8, 9, 9)),
    ("Joaquín", (10, 8, 8, 7)),
    ("Luján", (10, 9, 9, 10)),
]

{nombre: media(notas) for nombre, notas in datos}
```

```cmd
{'Marcos': 7.5, 'Joaquín': 8.25, 'Luján': 9.5}
```

:::